In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-25,France,124378.0,0.0,22781.0,0.0,0.0,0.0
0,2020-04-25,China,84005.0,0.0,4673.0,0.0,0.0,0.0
0,2020-04-25,Italy,195593.0,0.0,26417.0,0.0,0.0,0.0
0,2020-04-25,Spain,224483.0,0.0,22914.0,0.0,0.0,0.0
0,2020-04-25,United States,915431.0,0.0,53571.0,0.0,0.0,0.0
0,2020-04-25,World,2827149.0,0.0,202695.0,0.0,0.0,0.0
0,2020-04-25,United Kingdom,148341.0,0.0,20173.0,0.0,0.0,0.0
0,2020-04-25,Germany,154466.0,0.0,154466.0,0.0,0.0,0.0
0,2020-04-25,Iran,89240.0,0.0,5663.0,0.0,0.0,0.0
0,2020-04-25,Turkey,107465.0,0.0,2709.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-26,France,124378.0,124114.0,22781.0,22614.0,264.0,167.0
0,2020-04-26,China,84005.0,83909.0,4673.0,4636.0,96.0,37.0
0,2020-04-26,Italy,195593.0,195351.0,26417.0,26384.0,242.0,33.0
0,2020-04-25,Spain,224483.0,219764.0,22914.0,22524.0,4719.0,390.0
0,2020-04-26,United States,915431.0,939053.0,53571.0,53189.0,-23622.0,382.0
0,2020-04-26,World,2827149.0,2844712.0,202695.0,201315.0,-17563.0,1380.0
0,2020-04-26,United Kingdom,148341.0,148377.0,20173.0,20319.0,-36.0,-146.0
0,2020-04-26,Germany,154466.0,154175.0,154466.0,5640.0,291.0,148826.0
0,2020-04-26,Iran,89240.0,89328.0,5663.0,5650.0,-88.0,13.0
0,2020-04-26,Turkey,107465.0,107773.0,2709.0,2706.0,-308.0,3.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")